In [1]:
!pip3 install opencv-python dlib face_recognition numpy scikit-learn mtcnn keras-facenet torch joblib

In [2]:
import os
import cv2
from mtcnn import MTCNN
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras_facenet import FaceNet
import logging
import torch

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
# Suppress TensorFlow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # '0' = all logs, '1' = filter INFO, '2' = filter WARNING, '3' = filter ERROR
# Set logging level to ERROR to suppress lower level messages
logging.basicConfig(level=logging.ERROR)
# Alternatively set TensorFlow logging verbosity directly
tf.get_logger().setLevel('ERROR')
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('mtcnn').setLevel(logging.ERROR)

In [5]:
# Path containing all the the videos
video_path = "./data/recordings/"
videos = os.listdir(video_path)
videos

frame_output_folder = "./data/frames/"
augment_output_folder = "./data/augmented_frames/"

In [6]:
# Initialize MTCNN detector
detector = MTCNN()
embedder = FaceNet()

# Initialize ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

In [7]:
def convert_video_to_frames(label_name, video_path, output_dir,max_frames=200, min_face_width=100, min_face_height=100):
    video_capture = cv2.VideoCapture(video_path)
    
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
   
    # Set frame sampling rate dynamically based on total number of frames
    if total_frames <= max_frames:
        frame_interval = 1  # No need to skip frames if video is short
    else:
        frame_interval = total_frames // max_frames  # Adjust frame sampling rate
    
    
    frame_count = 0
    extracted_frames = 0
    face_count = 0
    while True:
        
        # Read a single frame from the video
        ret, frame = video_capture.read()
        if not ret or extracted_frames >= max_frames:
            break  # Stop if video ends or we have reached 
        
        
        frame_count += 1
        # Process every nth frame (e.g., every 10th frame)
        if frame_count % frame_interval == 0:
            # Convert the frame from BGR (OpenCV format) to RGB (for face_recognition)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Find all face locations in the current frame
            face_detections = detector.detect_faces(rgb_frame)

            for i, detection in enumerate(face_detections):
                x, y, w, h = detection["box"]

                if w < min_face_width or h < min_face_height:
                    continue

                face_image = frame[y : y + h, x : x + w]
                face_image_resized = cv2.resize(face_image, (160, 160))
                # Save the cropped face image
                face_file_name = f"{label_name}_aug_{i}.jpg"
                face_path = os.path.join(output_dir, face_file_name)
                cv2.imwrite(face_path, face_image_resized)
                face_count += 1
            
            extracted_frames += 1  # Only count frames with detected faces
            if frame_count % 10 == 0:
                print(f"Processed {frame_count}/{total_frames} frames, extracted {extracted_frames}/{max_frames} so far...")

    # Release the video capture object
    video_capture.release()
    print(f"Completed processing {frame_count} frames, extracted {extracted_frames} frames with {face_count} faces.")

In [8]:
def augment_images(label, input_dir, output_dir):
    final_output_dir = output_dir + "/" + label + "/"

    if not os.path.exists(final_output_dir):
        os.makedirs(final_output_dir)

    total_images = len(os.listdir(input_dir))
    max_augmentations = 5 if total_images < 100 else 2  # Reduce augmentations if too many images

    for idx, img_name in enumerate(os.listdir(input_dir), 1):
        img_path = os.path.join(input_dir, img_name)
        img = cv2.imread(img_path)

        if img is not None:
            # Ensure the image is in RGB format
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Reshape to a 4D tensor for the ImageDataGenerator
            img = img.reshape((1,) + img_rgb.shape)

            i = 0
            for batch in datagen.flow(
                img,
                batch_size=1,
                save_to_dir=final_output_dir,
                save_prefix="aug",
                save_format="jpg",
            ):
                i += 1
                if i >= max_augmentations:  # Generate 5 augmented images per original image
                    break
        print(f"Augmented {idx}/{total_images} images")
        
    print(f"Completed augmentation of {total_images} images.")                

## Convert videos into frames and extract faces from the frames

In [ ]:
for _vid in videos:
    
    if os.path.isfile(_vid)
        continue
    
    label_name = _vid.split(".")[0]
    student_video_path = video_path + _vid

    frame_output_dir = frame_output_folder + label_name
    if not os.path.exists(frame_output_dir):
        os.makedirs(frame_output_dir)
    convert_video_to_frames(label_name, student_video_path, frame_output_dir)

In [14]:
framed_images = os.listdir(frame_output_folder)
print(framed_images)

['shyam', '.DS_Store', 'surya', 'pugal', 'lakshmi']


## Augmenting the faces generated from the frames

In [ ]:
for _vid in videos:

    if not _vid.endswith(".mov"):
        continue
    
    label_name = _vid.split(".")[0]
    student_video_path = video_path + _vid

    frame_output_dir = frame_output_folder + label_name
    augment_output_dir = augment_output_folder
    if not os.path.exists(augment_output_dir):
        os.makedirs(augment_output_dir)

    augment_images(label_name, frame_output_dir, augment_output_dir)

In [10]:
agumented_images = os.listdir(augment_output_folder)
agumented_images

['shyam', 'surya', 'pugal', 'lakshmi']

In [ ]:
# Load dataset
def generate_embedding_and_label(
    folder,label_name,  embeddings_file="./model/face_embeddings.npy", labels_file="./model/face_labels.npy"
):

    # Load existing embeddings and labels if they exist
    if os.path.exists(embeddings_file) and os.path.exists(labels_file):
        embeddings = list(np.load(embeddings_file, allow_pickle=True))
        labels = list(np.load(labels_file, allow_pickle=True))
        print(f"Loaded {len(labels)} existing faces.")
    else:
        embeddings = []
        labels = []
        print("No existing embeddings found. Starting fresh.")

    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder, filename)

            image = cv2.imread(image_path)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            detections = detector.detect_faces(rgb_image)

            for detection in detections:
                x, y, width, height = detection["box"]
                face = rgb_image[y : y + height, x : x + width]
                # Use FaceNet to compute face embeddings
                face_embedding = embedder.embeddings([face])

                # Append the face embeddings
                embeddings.append(face_embedding[0])

                # Assume filename corresponds to the label (e.g., student name)
                labels.append(label_name)

    return embeddings, labels

In [11]:
for _img in agumented_images:
  label_name = _img.split(".")[0]
  augmented_frame_output_dir = augment_output_folder + label_name
  
  print(f"Processing {label_name}...")
  
  face_embeddings, face_labels = generate_embedding_and_label(augmented_frame_output_dir, _img)
  np.save("./model/face_embeddings.npy", face_embeddings)
  np.save("./model/face_labels.npy", face_labels)

FileNotFoundError: [Errno 2] No such file or directory: 'shyam'

In [ ]:
embeddings = (np.load("./model/face_embeddings.npy", allow_pickle=True))
labels = (np.load("./model/face_labels.npy", allow_pickle=True))

In [ ]:
embeddings.shape, labels.shape